In [ ]:
import numpy as np
import pandas as pd


In [ ]:
graph = Graph()
doc_ids = ['doc0', 'doc1', 'doc3', 'doc5', 'doc7', 'doc9']
for doc_id in doc_ids:
    graph.add_node(Node(doc_id))
edges = {
    ('doc0', 'doc9'): 0.8,
    ('doc1', 'doc3'): 0.7,
    ('doc7', 'doc3'): 0.2,
    ('doc5', 'doc7'): 0.9,
    ('doc5', 'doc3'): 0.6,
}
for edge in edges:
    graph.add_edge(graph.get_node(edge[0]), graph.get_node(edge[1]), edges[edge], directed=False)

In [ ]:
result_path = max_tsp_traverse(graph)

In [ ]:
print("Result path:", [node.get_doc_id() for node in result_path])

read the saved search results (in TREC format)

In [ ]:
import tqdm

In [ ]:


# Example usage:
file_path = '/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/bm25_search_results/chunk_88.result.txt'
result_dict = read_trec_results(file_path)





In [ ]:
len(result_dict)

In [ ]:
# read result from all chunks
result_dict = {}
for i in tqdm.tqdm(list(range(0, 89)) + ['search_fail_queries_added']):
    file_path = '/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/bm25_search_results/chunk_{i}.result.txt'.format(i=i)
    result_dict.update(read_trec_results(file_path))

print("Total number of queries:", len(result_dict))


In [ ]:
# count how many docs have itself as the top1 neighbor
count = 0
# length counts
lengths = []
for query_id, docs in result_dict.items():
    lengths.append(len(docs))
    if docs[0]['doc_id'] == query_id:
        count += 1
print("Number of queries that have itself as the top1 neighbor:", count, "percentage:", count/len(result_dict))
print("Average number of neighbors:", np.mean(lengths))

In [ ]:

# Example usage:
directory_path = '/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/train/'
all_file_ids = get_file_ids(directory_path)

# Print the result
print(f"Total number of files: {len(all_file_ids)}")

In [ ]:
sequence_set = set()
for i in range(0, 89):
    for j in range(0, 10000):
        sequence_set.add("{i}_{j}".format(i=i, j=j))

In [ ]:
# ensure that the two sets are equal
len(sequence_set - set(all_file_ids))
len(set(all_file_ids) - sequence_set)

In [ ]:
not_searched_query_ids = list()
for query_id in sequence_set:
    if query_id not in result_dict:
        not_searched_query_ids.append(query_id)
print("Number of queries not searched:", len(not_searched_query_ids))
print("Example query IDs:", not_searched_query_ids[:10])

to_write_data = []
not_searched_query_ids = sorted(not_searched_query_ids)
for docid in tqdm.tqdm(not_searched_query_ids, total=len(not_searched_query_ids)):
    chunk_id, seq_id = docid.split("_")
    base_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/queries"
    jsonl_file = os.path.join(base_path, "chunk_{}.jsonl".format(chunk_id))
    with open(jsonl_file, "r") as f:
        # directly go the line
        line = f.readlines()[int(seq_id)]
        data = json.loads(line)
        assert data["id"] == docid
        if len(data['title']) > 2000:
            print("problematic docid", docid)
            print(data['title'])
            print(len(data['title'].split()))
            continue
        to_write_data.append(data)

        # a slower version
        # for line in f:
        #     line = line.strip()
        #     if line:
        #         data = json.loads(line)
        #         if data["id"] == docid:
        #             # print(data)
        #             # print(len(data['title'].split()))
        #             to_write_data.append(data)
        #             break
# write to jsonl
with open(os.path.join(base_path, "chunk_search_fail_queries_added.jsonl"), "w") as f:
    for data in to_write_data:
        f.write(json.dumps(data) + "\n")

In [ ]:
# how to import?
import sys
sys.path.append('/home/aiops/zhuty/tinyllama/processing/graphs')

from graph import Graph, Node

In [ ]:
print("len of all_file_ids", len(all_file_ids))

In [ ]:
## Add the documents as nodes
graph = Graph()
for doc_id in all_file_ids:
    graph.add_node(Node(doc_id))

print("Number of nodes:", graph.get_node_count())
# add the top_k neighbors as edges
top_k = 5
for query_id, docs in tqdm.tqdm(result_dict.items()):
    for doc in docs[:top_k+1]:
        if doc['doc_id'] == query_id:
            # neighbor is the query itself
            continue
        graph.add_edge(graph.get_node(query_id), graph.get_node(doc['doc_id']), doc['score'], directed=True)



In [ ]:
result_dict['2_1578']

In [ ]:
# traverse the graph
result_path = max_tsp_traverse(graph)

In [ ]:
graph.get_node_neighbors('2_1578')

In [ ]:
graph.get_node_count()

In [ ]:
result_path = [node.get_doc_id() for node in result_path]
print("Result path:", result_path[:10])

In [ ]:
print("hellow")

In [ ]:
commands = []
for i in range(0, 89):
    commands.append(("head -n 10000 redpajama_20b_id_added/train/chunk_{i}.jsonl > redpajama_2b_id_added/train/chunk_{i}.jsonl".format(i=i)))
print(";".join(commands))

In [ ]:
num_cpus = 5
filenames = range(0, 23)

In [ ]:
import numpy as np
np.array_split(filenames, num_cpus)